In [1]:
import ibis
from ibis import _
import sys
sys.path.append("../cng-python/")
from utils import set_secrets, source_secrets

con = ibis.duckdb.connect(extensions=["spatial"])
set_secrets(con)
source_secrets(con)


In [2]:

t1 = con.read_parquet("s3://public-biodiversity/ca30x30/ca_areas.parquet").select(_.id, _.geom)
t2 = con.read_parquet("s3://public-fire/calfire-2023.parquet").select(_.geometry).rename(geom = "geometry")


In [ ]:
# geometries must be valid.  this will double-count overlaps
expr = (t1
 .left_join(t2, t1.geom.intersects(t2.geom))
 .group_by(t1.id, t1.geom)
 .agg(overlap_fraction = (t1.geom.intersection(t2.geom).area() / t1.geom.area()).sum().coalesce(0) )
)

ibis.to_sql(expr)

expr.to_parquet("ibis_overlap.parquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))